In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from batman import TransitModel
from astropy.io import fits
import astropy.units as u
from astropy.time import Time

import sys
sys.path.insert(0, '../')
from linea import CheopsLightCurve, Planet

p = Planet.from_name("WASP-189 b")

In [ ]:
def simulate_roll_angle(phase, time, phase_offset=0.1): 
    # Compute rough approximation to a varying parallactic angle
    H = 2 * np.pi * (phase - phase_offset)
    q = np.arctan2(np.sin(H) + 0.3 * (time - time[0]),
                   (np.tan(np.radians(0)) *
                    np.cos(np.radians(20)) -
                    np.sin(np.radians(20))*np.cos(H)))
    return np.degrees(q)

In [ ]:
np.random.seed(42)

n_visits = 4

for i in range(n_visits):

    cheops_orbit_min = 99.5  # minutes

    all_times = np.linspace(p.t0 + p.per * (0.5 + i) - 0.1 * p.per, 
                            p.t0 + p.per * (0.5 + i) + 0.1 * p.per, 2500)
    cheops_phase = (((p.t0 - all_times) * u.day) % 
                    (cheops_orbit_min * u.min) / 
                    (cheops_orbit_min * u.min)
                   ).to(u.dimensionless_unscaled).value

    phase_cutoff = 0.45
    bjd_time = all_times[cheops_phase > phase_cutoff]
    utc_time = Time(bjd_time, format='jd').isot
    mjd_time = Time(bjd_time, format='jd').mjd

    n_points = len(bjd_time)

    conta_lc = 10 * np.ones(n_points) + np.random.randn(n_points)
    conta_lc_err = np.ones(n_points) / 100
    status = np.zeros(n_points)
    event = np.zeros(n_points)
    dark = 10 + np.random.randn(n_points)
    background = 100 + np.random.randn(n_points)
    roll_angle = simulate_roll_angle(cheops_phase[cheops_phase > phase_cutoff], bjd_time)
    location_x = 512 * np.ones(n_points)
    location_y = 512 * np.ones(n_points)
    centroid_x = location_x[0] + 0.2 * np.random.randn(n_points)
    centroid_y = location_y[0] + 0.2 * np.random.randn(n_points)

    p.t_secondary = 0.5
    p.fp = 80e-6
    model = TransitModel(p, bjd_time, 
                         supersample_factor=3,
                         transittype='secondary',
                         exp_time=bjd_time[1] - bjd_time[0]
                         ).light_curve(p)

    flux = (
        -2 * (bjd_time - bjd_time.mean()) + 
        2 * background + 
        1 * conta_lc + 
        dark + 
        20 * roll_angle / 360 + 
        1 * (centroid_x - location_x)**2 + 
        1.5 * (centroid_y - location_y)**2 +
        0.1 * (centroid_x - location_x) +
        0.1 * (centroid_y - location_y) + 
        2e3 * dark.mean()
    ) * model

    fluxerr = np.std(flux) * np.ones(len(bjd_time))

    n_outliers = 50

    flux[np.random.randint(0, flux.shape[0], size=n_outliers)] += 3.5 * flux.std() * np.random.randn(n_outliers)

    plt.errorbar(bjd_time, flux, fluxerr, fmt='.', ecolor='silver')
    plt.plot(bjd_time, model * np.median(flux), 'r')
    plt.show()

    formatter = [
        ('UTC_TIME', '|S26', utc_time),
        ('MJD_TIME', '>f8', mjd_time),
        ('BJD_TIME', '>f8', bjd_time),
        ('FLUX', '>f8', flux),
        ('FLUXERR', '>f8', fluxerr),
        ('STATUS', '>i4', status),
        ('EVENT', '>i4', event),
        ('DARK', '>f8', dark),
        ('BACKGROUND', '>f8', background),
        ('CONTA_LC', '>f8', conta_lc),
        ('CONTA_LC_ERR', '>f8', conta_lc_err),
        ('ROLL_ANGLE', '>f8', roll_angle),
        ('LOCATION_X', '>f4', location_x),
        ('LOCATION_Y', '>f4', location_y),
        ('CENTROID_X', '>f4', centroid_x),
        ('CENTROID_Y', '>f4', centroid_y)
    ]

    ra = np.recarray((n_points, ), 
                     names=[name for name, _, _ in formatter], 
                     formats=[fmt for _, fmt, _ in formatter])

    for name, fmt, arr in formatter: 
        ra[name] = arr

    fits.writeto(f'../linea/data/example_wasp189_visit{i}.fits', ra, overwrite=True)